In [ ]:
# !pip install -q sklearn
# !pip install tensorflow tensorflow-gpu opencv-python matplotlib
# !pip list
# !pip install tensorflow-gpu
# !pip install -U pip keras tensorflow

In [ ]:
# %tensorflow_version 2.x
import cv2
import glob
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
annotation = pd.read_csv("/content/drive/MyDrive/annotation.csv")
annotation

In [ ]:
SIZE = 224

images = []
labels = []
dir_path = "/content/data"
num_imgs = len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))])
iteration_count = 1
for directory_path in glob.glob("/content/data/"):
  label = directory_path.split("\\")[-1]

  for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):

    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    images.append(img)

    labelType = annotation.loc[annotation['Image_ID'] == img_path.split("/content/data/")[1]]['Type']
    # print("Striped: ", img_path.split("/content/data/")[1])
    img_name = img_path.split("/content/data/")[1]
    if (iteration_count < num_imgs*0.7):
      if (labelType.values == 'cyberbullying'):

        shutil.move(img_path, f"/content/drive/MyDrive/train/cyberbullying/{img_name}")
      else:
        shutil.move(img_path, f"/content/drive/MyDrive/train/noncyberbullying{img_name}")
    elif (iteration_count >= num_imgs*0.7 and iteration_count < num_imgs*0.85):
      if (labelType.values == 'cyberbullying'):

        shutil.move(img_path, f"/content/drive/MyDrive/validation/cyberbullying/{img_name}")
      else:
        shutil.move(img_path, f"/content/drive/MyDrive/validation/noncyberbullying{img_name}")
    else:
      if (labelType.values == 'cyberbullying'):

        shutil.move(img_path, f"/content/drive/MyDrive/test/cyberbullying/{img_name}")
      else:
        shutil.move(img_path, f"/content/drive/MyDrive/test/noncyberbullying/{img_name}")
    iteration_count += 1


**Loading Data & Scale Data**

In [ ]:

train = ImageDataGenerator(rescale = 1/255)
validation = ImageDataGenerator(rescale = 1/255)
train_dataset = train.flow_from_directory('/content/drive/MyDrive/train/', 
                                          target_size = (SIZE,SIZE), 
                                          batch_size = 3, 
                                          class_mode = 'binary')
validation_dataset = validation.flow_from_directory('/content/drive/MyDrive/validation/', 
                                                    target_size = (SIZE,SIZE), 
                                                    batch_size = 3, 
                                                    class_mode = 'binary')


Found 205 images belonging to 2 classes.
Found 49 images belonging to 2 classes.


**Model Construction**

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(SIZE,SIZE,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir = 'Logs'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [ ]:

model_fit = model.fit(train_dataset, epochs = 12, callbacks = [tensorboard_callback])


In [ ]:
# from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
# pre = Precision()
# re = Recall()
# acc = BinaryAccuracy()
# print(len(test))
# for batchS in test.as_numpy_iterator(): 
#     X, y = batchS
#     yhat = model.predict(X)
#     pre.update_state(y, yhat)
#     re.update_state(y, yhat)
#     acc.update_state(y, yhat)

# print(pre.result(), re.result(), acc.result())
# model.evaluate(test)

In [ ]:
fig = plt.figure()
plt.plot(model_fit.history['accuracy'], color='teal', label='accuracy')
plt.plot(model_fit.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(model_fit.history['loss'], color='teal', label='loss')
plt.plot(model_fit.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()